In [9]:
%load_ext lab_black

# Run Agent against itself

In [10]:
!luxai-s2 main.py main.py -v 2 -s 101 -o replay.html

150: player_0 lost all factories
150: player_1 lost all factories
2.9819037914276123


## Just double click replay.html to open window in jupyter lab
This is just a note for above. The replays work well in tabs